In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_excel(
    "C:\\Users\\subha\\Downloads\\MERGED_6 Banks_20250901_to_20250902_Cateogry Flagging MF.xlsx"
)


In [ ]:
df = (
    df
    .sort_values(['Group'])
)


In [ ]:
df['Date'] = pd.to_datetime(
    df['Date'],
    errors='coerce',
    format='%Y-%m-%d'   # works for yy-mm-dd too
)

In [ ]:
df['day'] = df['Date'].dt.day
df['week'] = df['Date'].dt.isocalendar().week.astype(int)

# Create as Period first
df['month'] = df['Date'].dt.to_period('M')
df['quarter'] = df['Date'].dt.to_period('Q')


In [ ]:
df['month'] = df['month'].astype(str)
df['quarter'] = df['quarter'].astype(str)


In [ ]:
monthly_pattern = (
    df.groupby('Cleaned Narration')
      .agg(
          months=('month', 'nunique'),
          day_std=('day', 'std')
      )
)

monthly_candidates = monthly_pattern[
    (monthly_pattern['months'] >= 3) &
    (monthly_pattern['day_std'] <= 3)
].assign(periodicity='Monthly')


In [ ]:
df['weekday'] = df['Date'].dt.weekday

weekly_pattern = (
    df.groupby('Cleaned Narration')
      .agg(
          weeks=('week', 'nunique'),
          weekday_std=('weekday', 'std')
      )
)

weekly_candidates = weekly_pattern[
    (weekly_pattern['weeks'] >= 4) &
    (weekly_pattern['weekday_std'] == 0)
].assign(periodicity='Weekly')



In [ ]:
quarterly_pattern = (
    df.groupby('Cleaned Narration')
      .agg(
          quarters=('quarter', 'nunique')
      )
)

quarterly_candidates = quarterly_pattern[
    quarterly_pattern['quarters'] >= 2
].assign(periodicity='Quarterly')


In [12]:
periodicity_map = (
    pd.concat([
        monthly_candidates[['periodicity']],
        weekly_candidates[['periodicity']],
        quarterly_candidates[['periodicity']]
    ])
    .reset_index()                       # bring group key back
    .drop_duplicates(
        subset='Cleaned Narration',      # or narr_cluster / ledger
        keep='first'                     # priority order preserved
    )
    .set_index('Cleaned Narration')      # make it a proper map
)


In [ ]:
df['periodicity'] = df['Cleaned Narration'].map(
    periodicity_map['periodicity']
).fillna('Irregular')


In [ ]:
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')



In [ ]:

df_export = df.copy()
cols_to_drop = ['day', 'week', 'month', 'quarter','weekday']
df_export = df_export.drop(columns=cols_to_drop, errors='ignore')


In [18]:
 df_export.to_excel("final_output.xlsx", index=False)